In [ ]:
import yaml
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Uncomment if SimBMVtool not in directory
# import sys
# path_SimBMVtool = '<path_to_SimBMVtool>'
# sys.path.append(path_SimBMVtool)

from SimBMVtool.external_bmv_creator import ExternalBMVCreator
from SimBMVtool.toolbox import (
    get_data_store,
    get_skymaps_dict,
    plot_skymap_from_dict,
    get_dfobs_table
)

path_config = './config_real.yaml'

In [ ]:
# Uncomment if you haven't downloaded gammapy datasets and update the path in config_real.yaml
# It is needed for the catalogs

# !gammapy download datasets

# Example dataset

The example dataset is a set of pointlike DL3s of Large Zenith Angle observations of the Crab Nebula
You need to copy it, new index tables will be created with the correct paths

Use the destination path as input in the configuration file `config_real.yaml` with `path_data: <path_to_copy>`

The observation and hdu tables with the output background model paths will be stored there

In [ ]:
!mkdir -p ./crab_LZA_data
!cp -r /fefs/aswg/workspace/marie-sophie.carrasco/data/mc/DL3/AllSky/20230927_v0.10.4_crab_tuned/TestingDataset/dec_2276/crab_LZA/pointlike/globalgh090_pe200_thetacut_020/data/dl3_LST-1.Run12* ./crab_LZA_data

In [ ]:
# Check the files in the directory following config['paths']['path_data']
with open(path_config, 'r') as file:
    config = yaml.safe_load(file)
path_data = config["paths"]["path_data"]
obs_pattern = config["data"]["obs_pattern"]
source_name = config["source"]["catalog_name"]

data_store = get_data_store(path_data,obs_pattern)
dfobs_table = get_dfobs_table(data_store.obs_table)

# Look for the source in case there are several in the table.
# The objects are compared with lower case and no space to account for different names
is_source = dfobs_table.OBJECT.str.lower().str.replace(' ', '') == source_name.lower().replace(' ','')
display(dfobs_table.loc[is_source].reset_index())

In [ ]:
# You can check the data_store and the observations directly from the class object
realbmv = ExternalBMVCreator(path_config, real_data=True)

realbmv.load_observation_collection()
display(realbmv.data_store.obs_table)       # Or directly with realbmv.obs_table

In [ ]:
# Check the first loaded observation
iobs = 0
obs_id = int(realbmv.obs_collection[iobs].obs_id)

print(f"Obs id: {obs_id}")
realbmv.obs_collection[iobs].events.peek()

# You can access the file path with the obs_table stored as a pandas dataframe: realbmv.dfobs_table
# Usually the file paths are already in the data_store but not always, depending on how the data_store was obtained. 
# This is why the tool produces automatically the obs_table dataframe and stores the paths in it in case they are missing

print(f"Run {obs_id} file: {realbmv.dfobs_table.loc[obs_id, 'FILE_NAME']}")

In [ ]:
# Check the exclusion region that will be used for the background modeling
realbmv.plot_exclusion_mask()

# Perform background modeling or load the observations and models if the files already exist

do_baccmod = True
if do_baccmod: realbmv.do_baccmod()
else: 
    realbmv.load_observation_collection(from_index=True)
    realbmv.load_output_background_irfs()

# If you get an IndexError, check that you put the correct OBJECT name and observation pattern in the config file

In [ ]:
# Take a look at the output background model
realbmv.bkg_output_irf_collection[np.argwhere(realbmv.obs_ids == obs_id).ravel()[0]].peek()

In [ ]:
# Plot the model obtained for the first run for each energy bin
realbmv.plot_model(data='acceptance',irf='output', i_irf=iobs)

In [ ]:
# Change the exclusion region from here.
# In this example I am using the n_circles method which appends as many circle regions as needed
# The first circle is considered the one centered on the source

# Now I want a larger region on the Crab and an additionnal circle on Zeta Tauri

# Change radius for the first circle
realbmv.n_circles_radius[0] = 0.35

# Add radec coordinates and radius for the second circle
realbmv.n_circles_radec.append([84.4125, 21.1425])
realbmv.n_circles_radius.append(0.35)

# The new exclusion region is declared with the stored parameters instead of config dict
# The output file paths are updated accordingly
realbmv.init_exclusion_region(cfg_exclusion_region='stored_parameters', init_save_paths=True)

for i_circle in range(realbmv.n_circles):
    print(f"Circle {i_circle}: (ra,dec) = {realbmv.n_circles_radec[i_circle][0],realbmv.n_circles_radec[i_circle][1]} [°], radius = {realbmv.n_circles_radius[i_circle]}°")

realbmv.plot_exclusion_mask()

# Now do the modeling
realbmv.do_baccmod()
realbmv.bkg_output_irf_collection[iobs].peek()
realbmv.plot_model(data='acceptance',irf='output', i_irf=iobs)


# Uncomment if you want to revert to parameters from config file
# realbmv.init_exclusion_region(cfg_exclusion_region='stored_config')

In [ ]:
# Create a zenith binned collection of models

# zenith_bins: n_bins, np.array(bin_edges), 'baccmod', 'stored'
# 'baccmod' option will recompute the binning with the baccmod method
# 'stored' option will use already stored bins so you don't have to recompute every time

# The example config has zenith binning set on False, so you should see the same output for each zenith bin

realbmv.plot_zenith_binned_model(data='acceptance',irf='output', zenith_bins=2)

In [ ]:
# Set the zenith binning to True
# You can set up any modeling parameter from here.

realbmv.zenith_binning = True
realbmv.cos_zenith_binning_parameter_value = 2000
realbmv.do_baccmod()

# Plot the results.
realbmv.plot_zenith_binned_model(data='acceptance',irf='output', zenith_bins='baccmod')

In [ ]:
# The plotting function binned the models. 
# Now we can bin the observations to check the binning, with zenith_bins="stored" to avoid recalculating it

realbmv.create_zenith_binned_collections(collections=['observation'], zenith_bins="stored")

for coszd_bin_center in realbmv.cos_zenith_bin_centers:
    coszd_bin_center_zenith = np.rad2deg(np.acos(coszd_bin_center))
    obs_ids_in_coszd_bin = np.array(realbmv.zenith_binned_obs_collection[coszd_bin_center].ids).astype(int)
    print(f"zd = {coszd_bin_center_zenith:.1f}°: ", obs_ids_in_coszd_bin)

realbmv.plot_zenith_binned_data(data='livetime', per_wobble=True)

In [ ]:
# Here is an exemple of how to specify a run list and to exclude an array of obs_ids from it

runs_to_exclude = np.array([12161, 12162, 12289, 12290])
run_list = realbmv.all_obs_ids[~pd.Series(realbmv.all_obs_ids).isin(runs_to_exclude)]

realbmv.run_list = run_list

realbmv.zenith_binning = True
realbmv.cos_zenith_binning_parameter_value = 2000

realbmv.do_baccmod()

realbmv.create_zenith_binned_collections(collections=['observation'], zenith_bins="baccmod")

for coszd_bin_center in realbmv.cos_zenith_bin_centers:
    coszd_bin_center_zenith = np.rad2deg(np.acos(coszd_bin_center))
    obs_ids_in_coszd_bin = np.array(realbmv.zenith_binned_obs_collection[coszd_bin_center].ids).astype(int)
    print(f"zd = {coszd_bin_center_zenith:.1f}°: ", obs_ids_in_coszd_bin)
    
realbmv.plot_zenith_binned_data(data='livetime', per_wobble=True)

# Plot the results.
realbmv.plot_zenith_binned_model(data='acceptance',irf='output', zenith_bins='stored')

# In this case, I chose to create the binned collection first with the 'baccmod' option, then to plot the binned models with the stored bins
# Both ways are possible and independent. 
# With collections=['observation', 'output'] you can bin the both the observations and the output models without having to plot them

In [ ]:
# Plot only one zenith bin
icoszd_bin = 0
realbmv.plot_zenith_binned_model(data='acceptance',irf='output', i_bin=icoszd_bin)

# You can access the file list with the obs_table stored as a pandas dataframe
coszd_bin_center_zenith = np.rad2deg(np.acos(realbmv.cos_zenith_bin_centers[icoszd_bin]))
obs_ids_in_coszd_bin = np.array(realbmv.zenith_binned_obs_collection[realbmv.cos_zenith_bin_centers[icoszd_bin]].ids).astype(int)

print(f"Files in bin with zd = {coszd_bin_center_zenith:.1f}°: \n",
      realbmv.dfobs_table.loc[obs_ids_in_coszd_bin, 'FILE_NAME'].to_numpy())

# Skymaps

In [ ]:
# Uncomment and run if you need to reset the obs_collection loaded with the output models
# realbmv.run_list = run_list
# realbmv.load_observation_collection(from_index=True)

In [ ]:
# Get the full stacked dataset (this step can get quite long if you have a lot of statistics)
e_min, e_max = (realbmv.e_min, realbmv.e_max)
offset_max_dataset = realbmv.size_fov_acc
nbin_offset_dataset = 10 * realbmv.nbin_offset_acc
realbmv.axis_info_dataset = [e_min, e_max, offset_max_dataset, nbin_offset_dataset]

stacked_dataset = realbmv.get_stacked_dataset(bkg_method='ring', axis_info=realbmv.axis_info_dataset)

# Get and plot the skymaps
skymaps = get_skymaps_dict(stacked_dataset, realbmv.exclude_regions, realbmv.correlation_radius, realbmv.correlate_off, 'all')
print("Skymaps in dict: ", [key for key in skymaps.keys()])

# Use the toolbox to plot with some pre-formatting
plot_skymap_from_dict(skymaps, 'counts')

# Crop the map and/or reduce the size of the plot
plot_skymap_from_dict(skymaps, 'excess', crop_width=2.*u.deg, figsize=(6,6))

# We see that the excess map behaves badly. 
# This is related to the background model empty bins at larger offset for some energy bins.
# If you crop the map to a safe width then it becomes good again

plot_skymap_from_dict(skymaps, 'excess', crop_width=1.2 * u.deg, figsize=(6,6))

# Plot any map directly the usual way with gammapy
skymap = skymaps['significance_off']
skymap.cutout(position=realbmv.source_pos, width=1.7 * u.deg).smooth(0.01 * u.deg).plot(add_cbar=True, cmap='magma', stretch="linear")

In [ ]:
# Check the residuals distribution. 
# 'significance_all' and 'significance_off' need to be in skymaps_dict
realbmv.plot_residuals_histogram(skymaps_dict=skymaps)

# Now the skymaps are stored in the class object
realbmv.skymaps_dict.keys()

In [ ]:
# The histogrammed distribution should be close to a normal distribution with mean->0 and std->1
# Here it is not the case due to the empty bins in the background models for larger energies and/or larger offsets.
# To get good results you need to give energy range and offset max according to the model

In [ ]:
# Plot for a given energy range, following the energy bins for example
iEbin = 0
Ebin_edges = realbmv.Ebin_tuples[iEbin].data

e_min, e_max = Ebin_edges
print(f"E min, E max = {e_min:.1f}, {e_max:.1f} TeV")

stacked_sliced = stacked_dataset.slice_by_energy(e_min * u.TeV, e_max * u.TeV)
skymaps_sliced = get_skymaps_dict(stacked_dataset, realbmv.exclude_regions, realbmv.correlation_radius, realbmv.correlate_off, 'all')

plot_skymap_from_dict(skymaps_sliced, 'background', crop_width=1/np.sqrt(2)*realbmv.size_fov_acc, figsize=(5,5))

# You can compute and plot all the skymaps and the histogram at once for any given external MapDatasetOnOff
realbmv.plot_skymaps('ring', stacked_sliced)

# The skymaps dictionary is now stored in the class object if you want to plot a specific map with different parameters
plot_skymap_from_dict(realbmv.skymaps_dict, 'significance_off', crop_width=1/np.sqrt(2)*realbmv.size_fov_acc, figsize=(4,4))

# The given dataset is now stored in the class object
print(realbmv.stacked_dataset)

In [ ]:
# If you don't give a stacked dataset as input, the plot_skymaps method will automatically create one from the obs_collection with the desired parameters
e_min, e_max = (0.1*u.TeV, 0.7*u.TeV)
offset_max_dataset = 2.
realbmv.axis_info_dataset = [e_min, e_max, offset_max_dataset * u.deg, 10 * realbmv.nbin_offset_acc]

realbmv.plot_skymaps('ring')
# Now the histogram is better due to the safe offset_max used to produce the dataset

In [ ]:
# You can change the ring background parameters
internal_ring_radius = 0.35         # [°]
width = 0.1                         # [°]
realbmv.ring_bkg_param = [internal_ring_radius, width]

realbmv.plot_skymaps('ring')

In [ ]:
# Plot for each zenith bin with the FoV background maker
e_min, e_max = (2*u.TeV, 3*u.TeV)
offset_max_dataset = 1.5
realbmv.axis_info_dataset = [e_min, e_max, offset_max_dataset * u.deg, 10 * realbmv.nbin_offset_acc]

for icoszd_bin in range(len(realbmv.zenith_binned_obs_collection.keys())):
    # Get obs_ids in coszd bin
    coszd_bin_center_zenith = np.rad2deg(np.acos(coszd_bin_center))
    obs_ids_in_coszd_bin = np.array(realbmv.zenith_binned_obs_collection[realbmv.cos_zenith_bin_centers[icoszd_bin]].ids).astype(int)
    print(f"zd = {coszd_bin_center_zenith:.1f}°: ", obs_ids_in_coszd_bin)
    
    # Load only the observations in the coszd bin
    realbmv.run_list = obs_ids_in_coszd_bin
    realbmv.load_observation_collection(from_index=True)

    # Plot
    realbmv.plot_skymaps('FoV')
    plt.show()

# As an exercice you can try to change the parameters within the loop to get zenith-dependent skymaps, 
# like by changing the offset_max_dataset depending on the empty bins in the output model